# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
# ! wget https://dataset-ppm.s3.amazonaws.com/iris.csv

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')
data.rename(columns={'sepal.length':'sepal length', 'sepal.width':'sepal width', 'petal.length':'petal length', 'petal.width':'petal width'}, inplace=True)
data['variety'] = data['variety'].map({'Setosa':'Iris-setosa', 'Virginica':'Iris-virginica', 'Versicolor':'Iris-versicolor'})



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal length,sepal width,petal length,petal width,variety
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
print(data_uji.shape)
print(data_latih.shape)

(30, 5)
(120, 5)


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [6]:
label_uji = data_uji.pop('variety')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [7]:
def buat_trainingset(dataset):
    trainingset = {}
    kolom_kelas = dataset.columns[-1]
    print("Kolom kelas: ", kolom_kelas)
    list_kelas = dataset[kolom_kelas].value_counts()
    list_kelas = list_kelas.keys()
    print("Kelas unik: ", list_kelas)
    
    for kelas in list_kelas:
        data_temp = dataset.copy(deep=True)
        data_temp[kolom_kelas] = data_temp[kolom_kelas].map({kelas:1})
        data_temp[kolom_kelas] = data_temp[kolom_kelas].fillna(-1)
        trainingset[kelas] = data_temp
    
    return trainingset


Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training.

In [8]:
trainingset = buat_trainingset(data_latih)

Kolom kelas:  variety
Kelas unik:  Index(['Iris-versicolor', 'Iris-virginica', 'Iris-setosa'], dtype='object', name='variety')


Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [9]:
print(trainingset['Iris-versicolor'])

     sepal length  sepal width  petal length  petal width  variety
57            4.9          2.4           3.3          1.0      1.0
12            4.8          3.0           1.4          0.1     -1.0
81            5.5          2.4           3.7          1.0      1.0
106           4.9          2.5           4.5          1.7     -1.0
30            4.8          3.1           1.6          0.2     -1.0
..            ...          ...           ...          ...      ...
137           6.4          3.1           5.5          1.8     -1.0
123           6.3          2.7           4.9          1.8     -1.0
48            5.3          3.7           1.5          0.2     -1.0
68            6.2          2.2           4.5          1.5      1.0
66            5.6          3.0           4.5          1.5      1.0

[120 rows x 5 columns]


## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [10]:
def hitung_cost_gradient(W, X, Y, regularization):
    jarak = 1 - (Y * np.dot(X, W))
    dw = np.zeros(len(W))
    if max(0, jarak)==0:
        di = W
    else:
        di = W - (regularization * Y * X)
    dw += di
    return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [11]:
from sklearn.utils import shuffle

def sgd(data_latih, label_latih, learning_rate=1e-6, max_epoch=1000, regularization=10000):
    data_latih = data_latih.to_numpy()
    label_latih = label_latih.to_numpy()
    bobot = np.zeros(data_latih.shape[1])

    for epoch in range(1, max_epoch):
        X, Y = shuffle(data_latih, label_latih, random_state=101)
        for index, x in enumerate(X):
            delta = hitung_cost_gradient(bobot, x, Y[index], regularization)
            bobot = bobot - (learning_rate * delta)

    return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [12]:
def training(trainingset):
    list_kelas = trainingset.keys()
    W = {}
    for kelas in list_kelas:
        data_latih = trainingset[kelas]
        label_latih = data_latih[data_latih.columns[-1]]
        data_latih = data_latih[data_latih.columns[:-1]]
        W[kelas] = sgd(data_latih=data_latih, label_latih=label_latih)
    return W

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [13]:
W = training(trainingset)

Tampilkan isi variabel **W**

In [14]:
W

{'Iris-versicolor': array([ 0.87398409, -1.82939541,  1.16589559, -3.27004809]),
 'Iris-virginica': array([-3.85200894, -3.9357549 ,  5.91048345,  4.40903132]),
 'Iris-setosa': array([ 0.22207839,  0.64870486, -1.00917931, -0.46355615])}

## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [15]:
def testing(W, data_uji):
    prediksi = np.array([])
    for i in range(data_uji.shape[0]):
        y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
        prediksi = np.append(prediksi, y_prediksi)
    return prediksi

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [16]:
def testing_onevsrest(W,data_uji):
  
  list_kelas = W.keys()
  kelas_prediksi = [None] * data_uji.shape[0]
  
  for kelas in list_kelas:
    current_weight = W[kelas]
    current_pred = testing(W=current_weight, data_uji=data_uji)

    for i in range(len(current_pred)):
      if current_pred[i] == 1:
        if kelas_prediksi[i] == None:
          kelas_prediksi[i] = kelas
        else:
          kelas_prediksi == "N/A"
  
  return kelas_prediksi

In [20]:
label_prediksi = testing_onevsrest(W, data_uji)

Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [21]:
def accuracy(label_prediksi, label_uji):
    return sum(label_prediksi==label_uji)

In [22]:
accuracy = accuracy(label_prediksi, label_uji)
print(accuracy)

21


## Simpulan

Pada praktikum kali ini, dibuat sebuah model Support Vector Machine (SVM) pada dataset Iris dengan tipe 'One-vs-Rest'. Pada model SVM 'One-vs-Rest', permasalahan klasifikasi dibagi menjadi \frac{n*(n-1)}{2} permasalahan, dengan n menyatakan banyaknya kelas.
- Pada proses training, dibentuk 3 jenis training data untuk masing-masing kelas: target class disimbolkan 1 untuk semua data pada kelas tersebut, dan lainnya -1.
- Untuk masing-masing training data, dibentuk sebuah hyperplane yang mengklasifikasikan kelas tersebut dari kelas lainnya. Sehingga, dengan 3 kelas dan 3 training data, akan didapatkan 3 hyperplane berbeda. Untuk masing-masing hyperplane, weight tiap variable didapatkan dengan menjalankan fungsi minimasi _cost-function_ dengan _Stochastic Gradient Descent_ (SGD).
- Model akhir merupakan perpaduan/union ketiga hyperplane. Oleh karenanya, beberapa hyperplane memungkinkan adanya overlap area ataupun area yang tidak termasuk ke dalam ketiga hyperplane.
- Proses klasifikasi dilakukan dengan melakukan komparasi hasil prediksi masing-masing hyperplane dalam bentuk one-hot encoding. Suatu datapoint akan dimasukkan ke dalam kelas jika model memberikan label '1' untuk salah satu kelas. Akan tetapi, bisa ditemukan kejadian di mana sebuah datapoint masuk ke dalam dua atau lebih kelas dikarenakan adanya area overlap pada hyperplane ataupun datapoint tersebut tidak masuk ke kelas manapun.
- Hasil akhir pada model ini menunjukkan akurasi sebesar 70%, di mana 21 dari 30 data uji berhasil diklasifikasikan ke kelas yang benar.  